In [ ]:
import requests
base_url = "https://api.prolific.com/api/v1/"
headers = {
    'Authorization': '[SECRET TOKEN]',
    'Content-Type': 'application/json'
}
def api_query(request,headers={},params={},method="get"):
    if method == "get":
        response = requests.get(request, headers=headers, params=params)
    elif method == "post":
        response = requests.post(request, headers=headers, json=params)
    elif method == "patch":
        response = requests.patch(request, headers=headers, json=params)
    elif method == "delete":
        response = requests.delete(request, headers=headers, json=params)
    else:
        raise Exception(f"Invalid method: {method}")
    if 200 <= response.status_code < 300:
        return response.json()
    else:
      raise Exception(f"Request failed with status code: {response.status_code}, message: {response.text}")

In [ ]:
# Retreive your user info
print(api_query(base_url + "users/me/",headers))

In [ ]:
# Retrive info associated with all studies
print(api_query(base_url + "studies",headers))

In [ ]:
# Retrive info associated with just your unpublished or active studies
params = {"state": "(UNPUBLISHED|ACTIVE)"}
api_query(base_url + "studies",headers,params)

In [ ]:
# Get ids of all 'workspaces' you belong to:
api_query(base_url + "workspaces",headers)

In [ ]:
# Get ids of all projects in the workspace:
default_workspace = "[DEFAULT-WORKSPACE-ID]"
api_query(base_url + "workspaces/{}".format(default_workspace),headers)

In [ ]:
# Create a draft study
tutorial_project = '[PROJECT-ID]'
draft_study_params = {
    "name": "A Google Forms survey on music preferences",
    "internal_name": "API Google Forms music survey",
    "description": "In this survey, you'll be asked a few questions about your taste in music. The survey consists of three questions, and we anticipate that it can be completed in under three minutes. There are no requirements for taking part in this study -- simply answer the questions as best you can. Thanks for participating! NOTICE: There are no known risks, costs, or discomforts in this survey. We cannot and do not guarantee or promise that you will receive any benefits from this survey. Your participation in this experiment will take less than 3 minutes. You will be paid for your participation at the posted rate, consistent with online payment standards. If you have read this form and have decided to participate in this survey, please understand your participation is voluntary and you have the right to withdraw your consent or discontinue participation at any time without penalty or loss of benefits to which you are otherwise entitled. You have the right to refuse to answer particular questions. Your individual privacy will be maintained in all data collected from this survey.",
    "external_study_url": "[STUDY URL]",
    "prolific_id_option": "question",
    "completion_codes": [{'code' : "[COMPLETION CODE]",
                          'code_type' : "COMPLETED",
                          'action' : "MANUALLY_REVIEW"}],
    "total_available_places": 10,
    "estimated_completion_time": 3,
    "reward": 75,
    "project" : tutorial_project
}

In [ ]:
make_draft = api_query(base_url + "studies",headers,draft_study_params,method="post")
experimentid = make_draft['id']
experimentid

In [ ]:
# Retrieve a list of filters
filters = api_query(base_url + "filters",headers)['results']
[f['filter_id'] for f in filters][:5]

In [ ]:
[f for f in filters if f['filter_id'] == "current-country-of-residence"][0]

In [ ]:
# Update your study with one of the filters.
# NOTE: not all fields can be updated after the study has been published. See https://docs.prolific.com/docs/api-docs/public/#tag/Studies/operation/UpdateStudy
experiment_update = {
    'filters' : [{
        'filter_id' : "current-country-of-residence",
        'selected_values' : ["1"]
    }]
}
update_study = api_query(base_url + "studies/{}".format(experimentid),headers=headers,params=experiment_update,method="patch")
print("Updated study " + update_study['id'])

In [ ]:
# Run your study. DANGER ZONE: RUNS YOUR STUDY!
experiment_run = {
    "action": "PUBLISH"
    }
# # commented out for safety
# run_study = api_query(base_url + "studies/{}}/transition/".format(experimentid),headers=headers,params=experiment_run,method="post")
print("Running study " + run_study['id'])

In [ ]:
# Retrieve data associated with your study.
# demographic_data = api_query(base_url + "studies/{}/export/".format(experimentid),headers)